In [1]:
#import packages
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

In [2]:
#import packages
import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
import rtree  # supports geospatial join
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
from shapely.ops import nearest_points
from datetime import datetime as dt, date
sys.path.append('/Users/jackepstein/Documents/GitHub/wildfires-1001/code/functions/')
data_dir = '/Users/jackepstein/Documents/GitHub/wildfires-1001/data'
code_dir = '/Users/jackepstein/Documents/GitHub/wildfires-1001/code'

# Pull in main data frame

In [3]:
#pull in the target data frame and weather dictionary 
#make sure to change the pkl file name if needed
target_dict = {}
target_df = gpd.GeoDataFrame()
for i in np.arange(1, 3):
    target_dict[i] = pd.read_pickle(os.path.join(data_dir, f'clean_data/target_df_final_1123_newtargets_{i}.pkl')) 
    target_df = target_df.append(target_dict[i])


weather_dict_path = os.path.join(data_dir, 'clean_data/ERA_weather-data/ERA_rename_dictionary.pkl')

In [4]:
#load the naming dictionary
with open(weather_dict_path, 'rb') as handle:
    rename_dict = pickle.load(handle)

In [5]:
#rename the columns based on this dictionary
target_df.rename(columns = rename_dict, inplace = True)

In [6]:
#create lists of columns to drop and what our targets are
non_mod_cols = ['GRID_ID','month_id','MONTH','COUNTYFP','NAME','GRID_AREA','COUNTY_ARE','COUNTY_AREA',
                'geometry', 'adj_fire_count','adj_fire_bcount', 'Fire_area','Index','index']
bad_features = ['hist_p_time_1m', 'total_fire_days', 'hist_p_time_1y','month_id_old']
Y_cols = ['Y_bin', 'Y_fire_count', 'Y_fire_area_prop', 'Y_fire_class_size','Y_bin_new_fire_month',
          'Y_max_new_fire_size_month','Y_count_new_fires_month']

In [7]:
top_feats_rf = ['tot_area_fire_prev_1yr','2m_tmp_18hrs_5y','hist_fire_area_prop_1y','2m_tmp_18hrs_10y',
             '2m_tmp_18hrs_1y','hist_prop_area_fire_1m','2m_tmp_0hrs_5y','hist_fire_area_prop_1m',
             'tot_prcp_12hrs_10y','tot_prcp_6hrs_10y','hist_fire_area_prop_5y','2m_tmp_0hrs_1y','2m_tmp_0hrs_10y',
             'hist_fire_area_prop_10y','tot_prcp_18hrs_10y']

In [8]:
#convert floats from 64 to 32 for model
for col in target_df.columns:
    if target_df[col].dtypes == 'float64':
        target_df[col] = target_df[col].astype(np.float32)

# Split training and testing data

In [9]:
#generate training data set
#pre 2016
train_data = target_df[target_df['YEAR']<2016]
X_train = train_data.drop('YEAR', axis = 1)
#drop columns not used for modeling
for y in Y_cols + non_mod_cols + bad_features:
    try:
        X_train.drop(y, inplace = True, axis =1)
    except:
        pass
#set up target variable
Y_train_reg = train_data[['Y_fire_area_prop']]
Y_train_cl = train_data[['Y_bin_new_fire_month']]
Y_train_cl_size = train_data[['Y_max_new_fire_size_month']]

#generate testing data set - same logic as above
test_data = target_df[target_df['YEAR']>=2016]
X_test = test_data.drop('YEAR', axis = 1)
for y in Y_cols + non_mod_cols + bad_features:
    try:
        X_test.drop(y, inplace = True, axis =1)
    except:
        pass
Y_test_reg = test_data[['Y_fire_area_prop']]
Y_test_cl = test_data[['Y_bin_new_fire_month']]
Y_test_cl_size = test_data[['Y_max_new_fire_size_month']]

In [10]:
#check for any null values

#null vals array
null = np.zeros(len(X_train.columns))

for i in range(len(X_train.columns)):
    null[i] = X_train.loc[X_train[X_train.columns[i]].isna()].shape[0]
    
np.sum(null)

0.0

# Test Decision Tree -- baseline: OOB all features

In [11]:
#run tree
dt_clf_bl = DecisionTreeClassifier(criterion='entropy')
dt_clf_bl.fit(X_train, Y_train_cl)

DecisionTreeClassifier(criterion='entropy')

In [12]:
#get arrays
y_true = Y_test_cl.to_numpy().ravel()
y_preds_bl = dt_clf_bl.predict(X_test)
y_preds_prob_bl = dt_clf_bl.predict_proba(X_test)

print(roc_auc_score(y_true, y_preds_bl))

print(classification_report(y_true, y_preds_bl))

0.6022115000486431
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      5553
           1       0.30      0.31      0.30       783

    accuracy                           0.82      6336
   macro avg       0.60      0.60      0.60      6336
weighted avg       0.83      0.82      0.83      6336



# Test Decisiont Tree - hyperparameter tuning, all features

In [13]:
#generate lists of hyper params
leaf_base = 2
min_samples_leaf_values = []
for l in range(1,11):
    min_samples_leaf_values.append(leaf_base**l)
print(min_samples_leaf_values)

split_base = 2
min_samples_split_values = []
for l in range(4,12):
    param_int = int(split_base**l)
    min_samples_split_values.append(param_int)
    
print(min_samples_split_values)

[2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
[16, 32, 64, 128, 256, 512, 1024, 2048]


In [18]:
# Code here

#store a dictionary rather than the lists -- make it easier to plot after
recalls = {}
aucs = {}
#loop through the sample splits and create a list for each
for l in min_samples_leaf_values:
    recalls[l] = list()
    aucs[l] = list()
    
    #loop within one sample split, loop through the leaf sizes to get a different score
    for s in min_samples_split_values:
        
        clf = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=l, min_samples_split=s)
        clf = clf.fit(X_train, Y_train_cl)
        aucs[l].append(roc_auc_score(y_true, clf.predict(X_test)))
        recall = classification_report(y_true, clf.predict(X_test), output_dict=True)['1']['recall']
        recalls[l].append(recall)

/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [19]:
#check recalls
for k in recalls.keys():
    print(k,max(recalls[k]))     

2 0.31545338441890164
4 0.3282247765006386
8 0.3231162196679438
16 0.27330779054916987
32 0.26181353767560667
64 0.26053639846743293
128 0.26053639846743293
256 0.17752234993614305
512 0.11877394636015326
1024 0.0


In [20]:
#re-run on best parameters and get this feature importance
#use leaf size=2, split size=32
clf_full_best = DecisionTreeClassifier(criterion='entropy', 
                                       min_samples_leaf=2, min_samples_split=32).fit(X_train, Y_train_cl)

In [21]:
#get features
features_clf = pd.DataFrame()
features_clf['col'] = X_test.columns
features_clf['feature_importance'] = clf_full_best.feature_importances_
top_15_feats = features_clf.sort_values(by=['feature_importance'], ascending=False).head(15)['col'].to_list()
fea = features_clf.sort_values(by=['feature_importance'], ascending=False)
fea.to_csv('featureimportance_DT.csv')

# Test Decision Tree -- top 15 features (based on baseline)

In [22]:
#generate slimmed training data set
X_train_topdf = X_train[top_15_feats]
X_test_topdf = X_test[top_15_feats]


#generate slimmed training data set
X_train_toprf = X_train[top_feats_rf]
X_test_toprf = X_test[top_feats_rf]

In [23]:
# Code here

#store a dictionary rather than the lists -- make it easier to plot after
recalls_top = {}
#loop through the sample splits and create a list for each
for l in min_samples_leaf_values:
    recalls_top[l] = list()
    
    #loop within one sample split, loop through the leaf sizes to get a different score
    for s in min_samples_split_values:
        
        clf = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=l, min_samples_split=s)
        clf = clf.fit(X_train_topdf, Y_train_cl)
        recall = classification_report(y_true, clf.predict(X_test_topdf), output_dict=True)['1']['recall']
        recalls_top[l].append(recall)

/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [24]:
#check recalls
for k in recalls_top.keys():
    print(k,max(recalls_top[k]))
    
recalls_top[2]    

2 0.24265644955300128
4 0.26181353767560667
8 0.2962962962962963
16 0.21583652618135377
32 0.22094508301404853
64 0.21583652618135377
128 0.21583652618135377
256 0.1966794380587484
512 0.11877394636015326
1024 0.0


[0.22988505747126436,
 0.24265644955300128,
 0.21583652618135377,
 0.210727969348659,
 0.12260536398467432,
 0.14559386973180077,
 0.21583652618135377,
 0.0]

# Test Decision Tree -- top 15 features (based on RF)

In [25]:
# Code here

#store a dictionary rather than the lists -- make it easier to plot after
recalls_top2 = {}
#loop through the sample splits and create a list for each
for l in min_samples_leaf_values:
    recalls_top2[l] = list()
    
    #loop within one sample split, loop through the leaf sizes to get a different score
    for s in min_samples_split_values:
        
        clf = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=l, min_samples_split=s)
        clf = clf.fit(X_train_toprf, Y_train_cl)
        recall = classification_report(y_true, clf.predict(X_test_toprf), output_dict=True)['1']['recall']
        recalls_top2[l].append(recall)

/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jackepstein/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [26]:
#check recalls
for k in recalls_top.keys():
    print(k,max(recalls_top[k]))
    
    
recalls_top[2]    

2 0.24265644955300128
4 0.26181353767560667
8 0.2962962962962963
16 0.21583652618135377
32 0.22094508301404853
64 0.21583652618135377
128 0.21583652618135377
256 0.1966794380587484
512 0.11877394636015326
1024 0.0


[0.22988505747126436,
 0.24265644955300128,
 0.21583652618135377,
 0.210727969348659,
 0.12260536398467432,
 0.14559386973180077,
 0.21583652618135377,
 0.0]

# Test Multi-Class Trees

Based on results above, will use all features and winning hyperparameters (min_leaf=2, min_split=32)

In [27]:
#fit the model
clf_size = DecisionTreeClassifier(min_samples_leaf=2, min_samples_split=32, criterion='entropy')
clf_size.fit(X_train, Y_train_cl_size)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2,
                       min_samples_split=32)

In [28]:
#turn truth and predictions into arrays
y_test_size_arr = Y_test_cl_size.to_numpy().ravel()
preds_size = clf_size.predict(X_test)


print(confusion_matrix(y_test_size_arr, preds_size))


print(classification_report(y_test_size_arr, preds_size))

[[5274  133   64   82]
 [ 329   31   13   26]
 [ 129   12    4   19]
 [ 162   14    6   38]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      5553
           1       0.16      0.08      0.11       399
           2       0.05      0.02      0.03       164
           3       0.23      0.17      0.20       220

    accuracy                           0.84      6336
   macro avg       0.33      0.31      0.31      6336
weighted avg       0.80      0.84      0.82      6336



In [29]:
clf_size.feature_importances_
#get features
features_clf_size = pd.DataFrame()
features_clf_size['col'] = X_test.columns
features_clf_size['feature_importance'] = clf_size.feature_importances_
feats = features_clf_size.sort_values(by=['feature_importance'], ascending=False)
feats.to_csv('featureimportance_DT_size.csv')